In [1]:
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

import matplotlib.pyplot as plt

import os
import sys

from IPython.display import display, Image

from scipy import ndimage

import random
import cv2 
from sklearn.model_selection import train_test_split
%matplotlib inline
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

def show_img(img):
    fig = plt.gcf()
    fig.set_size_inches(18.5, 10.5)
    if len(img.shape) == 3:
        plt.imshow(img)
    else:
        img2 = img
        plt.imshow(img2, cmap='gray')
        plt.show()

# Get three channel data

In [2]:
%load_ext autoreload
%autoreload 2
from utils.preprocess_utils import *

# Try scratched Vgg

In [3]:
import numpy as np
import os
import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Activation,BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.metrics import categorical_accuracy
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras import regularizers
import keras.utils as ku
from keras import backend as K
from keras import optimizers
import tensorflow as tf
import pickle
import matplotlib.pyplot as ply

Using TensorFlow backend.


In [4]:
SHAPE = (64, 64)
CHANNEL = 3
    
data = generate_data_set_for_training(shape=SHAPE, channel=CHANNEL)
# Extract Train, val from data
X_train = data['X_train']
X_val = data['X_val']
# Format as the input of VGG
y_train = list(data['y_train'].T)
y_val = list(data['y_val'].T)

dataset: (86345, 64, 64, 3)
labels: (86345, 6)
dataset: (13066, 64, 64, 3)
labels: (13066, 6)


In [5]:
def get_lr_metric(optimizer):
    def lr(y_true, y_pred):
        return optimizer.lr
    return lr

In [6]:
_,row, col,channel = X_train.shape
digLen = 5 # including category 0
numDigits = 11
epochs = 75
batch_size = 64

optim = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
# optim = optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0)
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
tf.Session(config = config)

input = keras.Input(shape=(row,col,channel), name='customModel')
M = Conv2D(16,(3,3),activation='relu',padding='same',name = 'conv_16_1')(input)
M = Conv2D(16,(3, 3), activation ='relu', padding='same',name = 'conv_16_2')(M)
M = BatchNormalization(axis=-1)(M)
M = MaxPooling2D(pool_size=(2, 2))(M)

M = Conv2D(32, (3, 3), activation ='relu', padding='same', name = 'conv2_32_01')(M)
M = Conv2D(32, (3, 3), activation ='relu', padding='same', name = 'conv2_32_02')(M)
M = BatchNormalization(axis=-1)(M)
M = MaxPooling2D(pool_size=(2, 2))(M)
M = Dropout(0.5)(M)

M = Conv2D(48, (3, 3), activation ='relu', padding='same', name = 'conv2_48_01')(M)
M = Conv2D(48, (3, 3), activation ='relu', padding='same', name = 'conv2_48_02')(M)
M = BatchNormalization(axis=-1)(M)
M = MaxPooling2D(pool_size=(2, 2))(M)

M = Conv2D(64, (3, 3), activation ='relu', padding='same',name = 'conv2_64_1')(M)
M = Conv2D(64, (3, 3), activation ='relu', padding='same', name = 'conv2_64_2')(M)
M = Conv2D(64, (3, 3), activation ='relu', padding='same',name = 'conv2_64_3')(M)
M = BatchNormalization(axis=-1)(M)
M = MaxPooling2D((2, 2), strides= 1)(M)

M = Conv2D(128, kernel_size=(5, 5), activation='relu', padding='same',name = 'conv2_128_1')(M)
M = Conv2D(128, kernel_size=(5, 5), activation='relu', padding='same',name = 'conv2_128_2')(M)
M = Conv2D(128, kernel_size=(5, 5), activation='relu', padding='same',name = 'conv2_128_3')(M)
M = BatchNormalization(axis=-1)(M)
M = MaxPooling2D(pool_size=(2, 2),strides = 1)(M)

M = Conv2D(256, kernel_size=(5, 5), activation='relu', padding='same',name = 'conv2_128_5')(M)
M = Conv2D(256, kernel_size=(5, 5), activation='relu', padding='same',name = 'conv2_128_6')(M)
M = BatchNormalization(axis=-1)(M)
M = MaxPooling2D(pool_size=(2, 2),strides = 1)(M)
M = Dropout(0.5)(M)

M = Conv2D(256, (5, 5), activation='relu', padding='same',name = 'conv256_1')(M)
M = Conv2D(256, (5, 5), activation='relu', padding='same',name = 'conv256_2')(M)
M = Conv2D(256, (5, 5), activation='relu', padding='same',name = 'conv256_3')(M)
            # kernel_regularizer=regularizers.l2(0.01),
            # activity_regularizer=regularizers.l1(0.01))(M)
M = BatchNormalization(axis=-1)(M)
M = MaxPooling2D((2, 2), strides= 1)(M)

M = Conv2D(512, (5, 5), activation='relu', padding='same',name = 'conv2_512_1')(M)
M = Conv2D(512, (5, 5), activation='relu', padding='same',name = 'conv2_512_2')(M)
M = BatchNormalization(axis=-1)(M)
M = MaxPooling2D(pool_size=(2, 2),strides= 1)(M)
M = Dropout(0.25)(M)
# M = keras.layers.BatchNormalization(axis=-1)(M)

Mout = Flatten()(M)
Mout = Dense(2048, activation='relu', name = 'FC1_2048')(Mout)
Mout = Dense(1024, activation='relu', name = 'FC1_1024')(Mout)
Mout = Dense(1024, activation='relu', name = 'FC2_1024')(Mout)
# Mout = Dropout(0.5)(Mout)

numd_SM = Dense(digLen,    activation='softmax',name = 'num')(Mout)
dig1_SM = Dense(numDigits, activation='softmax',name = 'dig1')(Mout)
dig2_SM = Dense(numDigits, activation='softmax',name = 'dig2')(Mout)
dig3_SM = Dense(numDigits, activation='softmax',name = 'dig3')(Mout)
dig4_SM = Dense(numDigits, activation='softmax',name = 'dig4')(Mout)
numB_SM = Dense(2,         activation='softmax',name = 'nC')(Mout)
out = [numd_SM, dig1_SM ,dig2_SM, dig3_SM, dig4_SM, numB_SM]

svhnModel = keras.Model(inputs = input, outputs = out)

lr_metric = get_lr_metric(optim)
svhnModel.compile(loss = 'sparse_categorical_crossentropy', #ceLoss ,
                  optimizer= optim,
                  metrics=  ['accuracy']) #[])
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss',
                                              factor = 0.1,
                                              verbose = 1,
                                              patience= 2,
                                              cooldown= 1,
                                              min_lr = 0.00001)
svhnModel.summary()

callback = []
checkpointer = keras.callbacks.ModelCheckpoint(filepath='saved_models/designedBGRClassifier.hdf5',
                                               monitor='loss',
                                               save_best_only=True,
                                               verbose=2)
tb = keras.callbacks.TensorBoard(log_dir = 'logs',
                                  write_graph = True,
                                  batch_size = batch_size,
                                  write_images = True)
es = keras.callbacks.EarlyStopping(monitor= 'loss',  #'dig1_loss',
                                   min_delta=0.000001,
                                   patience=5,
                                   verbose=1,
                                   mode='auto')
callback.append(tb)
callback.append(es)
callback.append(checkpointer)
callback.append(reduce_lr)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
customModel (InputLayer)        (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
conv_16_1 (Conv2D)              (None, 64, 64, 16)   448         customModel[0][0]                
__________________________________________________________________________________________________
conv_16_2 (Conv2D)              (None, 64, 64, 16)   2320        conv_16_1[0][0]                  
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 64, 64, 16)   64          conv_16_2[0][0]                  
__________________________________________________________________________________________________
max_poolin

In [7]:
designHist = svhnModel.fit(x = X_train,
                          y = y_train,
                          batch_size = batch_size,
                          epochs = epochs,
                          verbose=1,
                          shuffle = True,
                          validation_data = (X_val, y_val),
                          callbacks= callback)

Train on 69076 samples, validate on 17269 samples
Epoch 1/75
69076/69076 [==============================] - 162s 2ms/step - loss: 3.5808 - num_loss: 0.7027 - dig1_loss: 1.0480 - dig2_loss: 0.9620 - dig3_loss: 0.4927 - dig4_loss: 0.1048 - nC_loss: 0.2706 - num_acc: 0.7648 - dig1_acc: 0.6789 - dig2_acc: 0.7108 - dig3_acc: 0.8972 - dig4_acc: 0.9849 - nC_acc: 0.9057 - val_loss: 4.7780 - val_num_loss: 1.0115 - val_dig1_loss: 1.3837 - val_dig2_loss: 1.1193 - val_dig3_loss: 0.5255 - val_dig4_loss: 0.0971 - val_nC_loss: 0.6409 - val_num_acc: 0.7546 - val_dig1_acc: 0.6833 - val_dig2_acc: 0.7012 - val_dig3_acc: 0.8959 - val_dig4_acc: 0.9865 - val_nC_acc: 0.9080

Epoch 00001: loss improved from inf to 3.58076, saving model to saved_models/designedBGRClassifier.hdf5
Epoch 2/75
69076/69076 [==============================] - 129s 2ms/step - loss: 2.9905 - num_loss: 0.4987 - dig1_loss: 0.9267 - dig2_loss: 0.8845 - dig3_loss: 0.4333 - dig4_loss: 0.0931 - nC_loss: 0.1541 - num_acc: 0.8188 - dig1_acc: 0

69076/69076 [==============================] - 95s 1ms/step - loss: 1.8797 - num_loss: 0.1102 - dig1_loss: 0.6205 - dig2_loss: 0.7661 - dig3_loss: 0.2977 - dig4_loss: 0.0548 - nC_loss: 0.0305 - num_acc: 0.9653 - dig1_acc: 0.7797 - dig2_acc: 0.7195 - dig3_acc: 0.8983 - dig4_acc: 0.9858 - nC_acc: 0.9905 - val_loss: 1.8421 - val_num_loss: 0.0988 - val_dig1_loss: 0.5886 - val_dig2_loss: 0.7743 - val_dig3_loss: 0.3034 - val_dig4_loss: 0.0475 - val_nC_loss: 0.0294 - val_num_acc: 0.9686 - val_dig1_acc: 0.7903 - val_dig2_acc: 0.7185 - val_dig3_acc: 0.8973 - val_dig4_acc: 0.9865 - val_nC_acc: 0.9902

Epoch 00012: loss improved from 1.89434 to 1.87970, saving model to saved_models/designedBGRClassifier.hdf5
Epoch 13/75
69076/69076 [==============================] - 97s 1ms/step - loss: 1.8673 - num_loss: 0.1092 - dig1_loss: 0.6115 - dig2_loss: 0.7645 - dig3_loss: 0.2976 - dig4_loss: 0.0543 - nC_loss: 0.0302 - num_acc: 0.9658 - dig1_acc: 0.7837 - dig2_acc: 0.7200 - dig3_acc: 0.8983 - dig4_acc: 0.

69076/69076 [==============================] - 97s 1ms/step - loss: 1.7358 - num_loss: 0.0968 - dig1_loss: 0.5298 - dig2_loss: 0.7373 - dig3_loss: 0.2943 - dig4_loss: 0.0524 - nC_loss: 0.0252 - num_acc: 0.9707 - dig1_acc: 0.8165 - dig2_acc: 0.7333 - dig3_acc: 0.8984 - dig4_acc: 0.9857 - nC_acc: 0.9922 - val_loss: 1.7094 - val_num_loss: 0.0969 - val_dig1_loss: 0.4990 - val_dig2_loss: 0.7354 - val_dig3_loss: 0.3034 - val_dig4_loss: 0.0470 - val_nC_loss: 0.0277 - val_num_acc: 0.9711 - val_dig1_acc: 0.8267 - val_dig2_acc: 0.7372 - val_dig3_acc: 0.8964 - val_dig4_acc: 0.9852 - val_nC_acc: 0.9907

Epoch 00023: loss improved from 1.75255 to 1.73582, saving model to saved_models/designedBGRClassifier.hdf5
Epoch 24/75
69076/69076 [==============================] - 96s 1ms/step - loss: 1.7161 - num_loss: 0.0959 - dig1_loss: 0.5211 - dig2_loss: 0.7276 - dig3_loss: 0.2951 - dig4_loss: 0.0521 - nC_loss: 0.0243 - num_acc: 0.9714 - dig1_acc: 0.8195 - dig2_acc: 0.7365 - dig3_acc: 0.8980 - dig4_acc: 0.

69076/69076 [==============================] - 97s 1ms/step - loss: 1.5607 - num_loss: 0.0853 - dig1_loss: 0.4585 - dig2_loss: 0.6550 - dig3_loss: 0.2916 - dig4_loss: 0.0490 - nC_loss: 0.0214 - num_acc: 0.9753 - dig1_acc: 0.8475 - dig2_acc: 0.7612 - dig3_acc: 0.8982 - dig4_acc: 0.9857 - nC_acc: 0.9930 - val_loss: 1.5678 - val_num_loss: 0.0958 - val_dig1_loss: 0.4444 - val_dig2_loss: 0.6500 - val_dig3_loss: 0.3009 - val_dig4_loss: 0.0499 - val_nC_loss: 0.0267 - val_num_acc: 0.9717 - val_dig1_acc: 0.8555 - val_dig2_acc: 0.7645 - val_dig3_acc: 0.8976 - val_dig4_acc: 0.9856 - val_nC_acc: 0.9917

Epoch 00034: loss improved from 1.57753 to 1.56069, saving model to saved_models/designedBGRClassifier.hdf5
Epoch 35/75
69076/69076 [==============================] - 96s 1ms/step - loss: 1.5456 - num_loss: 0.0847 - dig1_loss: 0.4505 - dig2_loss: 0.6483 - dig3_loss: 0.2916 - dig4_loss: 0.0496 - nC_loss: 0.0210 - num_acc: 0.9756 - dig1_acc: 0.8524 - dig2_acc: 0.7622 - dig3_acc: 0.8981 - dig4_acc: 0.

69076/69076 [==============================] - 96s 1ms/step - loss: 1.3417 - num_loss: 0.0694 - dig1_loss: 0.3648 - dig2_loss: 0.5590 - dig3_loss: 0.2846 - dig4_loss: 0.0469 - nC_loss: 0.0170 - num_acc: 0.9795 - dig1_acc: 0.8805 - dig2_acc: 0.7933 - dig3_acc: 0.9002 - dig4_acc: 0.9859 - nC_acc: 0.9946 - val_loss: 1.3771 - val_num_loss: 0.0920 - val_dig1_loss: 0.3526 - val_dig2_loss: 0.5568 - val_dig3_loss: 0.3021 - val_dig4_loss: 0.0454 - val_nC_loss: 0.0282 - val_num_acc: 0.9735 - val_dig1_acc: 0.8881 - val_dig2_acc: 0.7986 - val_dig3_acc: 0.8999 - val_dig4_acc: 0.9863 - val_nC_acc: 0.9910

Epoch 00045: loss improved from 1.36728 to 1.34172, saving model to saved_models/designedBGRClassifier.hdf5
Epoch 46/75
69076/69076 [==============================] - 96s 1ms/step - loss: 1.3300 - num_loss: 0.0717 - dig1_loss: 0.3603 - dig2_loss: 0.5478 - dig3_loss: 0.2843 - dig4_loss: 0.0474 - nC_loss: 0.0186 - num_acc: 0.9786 - dig1_acc: 0.8822 - dig2_acc: 0.7975 - dig3_acc: 0.9008 - dig4_acc: 0.

69076/69076 [==============================] - 97s 1ms/step - loss: 1.1206 - num_loss: 0.0576 - dig1_loss: 0.2825 - dig2_loss: 0.4657 - dig3_loss: 0.2557 - dig4_loss: 0.0449 - nC_loss: 0.0141 - num_acc: 0.9828 - dig1_acc: 0.9093 - dig2_acc: 0.8289 - dig3_acc: 0.9091 - dig4_acc: 0.9857 - nC_acc: 0.9957 - val_loss: 1.2025 - val_num_loss: 0.0857 - val_dig1_loss: 0.2920 - val_dig2_loss: 0.4794 - val_dig3_loss: 0.2738 - val_dig4_loss: 0.0449 - val_nC_loss: 0.0267 - val_num_acc: 0.9764 - val_dig1_acc: 0.9105 - val_dig2_acc: 0.8325 - val_dig3_acc: 0.9065 - val_dig4_acc: 0.9859 - val_nC_acc: 0.9922

Epoch 00056: loss improved from 1.17420 to 1.12055, saving model to saved_models/designedBGRClassifier.hdf5
Epoch 57/75
69076/69076 [==============================] - 96s 1ms/step - loss: 1.1123 - num_loss: 0.0564 - dig1_loss: 0.2803 - dig2_loss: 0.4634 - dig3_loss: 0.2540 - dig4_loss: 0.0447 - nC_loss: 0.0136 - num_acc: 0.9839 - dig1_acc: 0.9107 - dig2_acc: 0.8299 - dig3_acc: 0.9099 - dig4_acc: 0.

69076/69076 [==============================] - 97s 1ms/step - loss: 1.0826 - num_loss: 0.0546 - dig1_loss: 0.2715 - dig2_loss: 0.4495 - dig3_loss: 0.2497 - dig4_loss: 0.0444 - nC_loss: 0.0129 - num_acc: 0.9843 - dig1_acc: 0.9113 - dig2_acc: 0.8367 - dig3_acc: 0.9111 - dig4_acc: 0.9860 - nC_acc: 0.9961 - val_loss: 1.1785 - val_num_loss: 0.0842 - val_dig1_loss: 0.2856 - val_dig2_loss: 0.4693 - val_dig3_loss: 0.2676 - val_dig4_loss: 0.0451 - val_nC_loss: 0.0267 - val_num_acc: 0.9763 - val_dig1_acc: 0.9134 - val_dig2_acc: 0.8366 - val_dig3_acc: 0.9072 - val_dig4_acc: 0.9860 - val_nC_acc: 0.9922

Epoch 00067: loss did not improve from 1.07894
Epoch 68/75
69076/69076 [==============================] - 97s 1ms/step - loss: 1.0826 - num_loss: 0.0542 - dig1_loss: 0.2715 - dig2_loss: 0.4495 - dig3_loss: 0.2500 - dig4_loss: 0.0448 - nC_loss: 0.0126 - num_acc: 0.9841 - dig1_acc: 0.9126 - dig2_acc: 0.8356 - dig3_acc: 0.9109 - dig4_acc: 0.9859 - nC_acc: 0.9960 - val_loss: 1.1779 - val_num_loss: 0.08

In [11]:
%load_ext autoreload
%autoreload 2
from utils.model_evaluation_utils import evaluate_model_performance
modName = 'designed_model'
evaluate_model_performance(fitted_model=designHist, 
                           model_name = modName,
                           data=data,
                           model=svhnModel
                          )

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Train loss: 0.9507963826754848
number_of_digits digit1 digit2 digit3 digit4
Train accuracy: [98.74486073310557, 92.90057328160287, 85.60136661068968, 91.34576408593433, 98.60009265157218, 99.77560947362325]
Train sequence accuracy: 80.0958364699751
Validation loss: 1.1678723588553928
number_of_digits digit1 digit2 digit3 digit4
Validation accuracy: [97.62001273959117, 91.48763680583704, 84.06972030806648, 90.8738201401355, 98.61601714054086, 99.24720597602641]
Validation sequence accuracy: 78.06474028606173
Test loss: 2.521751479790146
number_of_digits digit1 digit2 digit3 digit4
Test accuracy: [95.73702739935712, 76.44267564671668, 61.24292055717129, 87.32588397367212, 99.12750650543394, 99.24996173274147]
Test sequence accuracy: 47.09168835144651


{'train_acc': [98.74486073310557,
  92.90057328160287,
  85.60136661068968,
  91.34576408593433,
  98.60009265157218,
  99.77560947362325],
 'test_acc': [95.73702739935712,
  76.44267564671668,
  61.24292055717129,
  87.32588397367212,
  99.12750650543394,
  99.24996173274147],
 'val_acc': [97.62001273959117,
  91.48763680583704,
  84.06972030806648,
  90.8738201401355,
  98.61601714054086,
  99.24720597602641],
 'train_seq_acc': 80.0958364699751,
 'test_seq_acc': 47.09168835144651,
 'val_seq_acc': 78.06474028606173,
 'train_score': [0.9507963826754848,
  0.04283636021068769,
  0.2223040749370617,
  0.3976787031656828,
  0.2371753952157807,
  0.04253297603639911,
  0.0082688738790532,
  0.9874486073310557,
  0.9290057328160287,
  0.8560136661068968,
  0.9134576408593433,
  0.9860009265157218,
  0.9977560947362325],
 'test_score': [2.521751479790146,
  0.16193167699625513,
  0.7790317756146508,
  1.1408709731781517,
  0.37813642449060003,
  0.03367830629113174,
  0.028102322761237586,
 